In [ ]:
### This notebook is intended to let a new lab member quickly create the checkerboard plots that we use in the lab. 

## Once you enter the correct Beiwe ID, you should be able to create a somewhat decent looking pdf output by running the following in shell:

# jupyter nbconvert --to pdf --TemplateExporter.exclude_input=True  "data_volume_summaries_template.ipynb"

## make sure you replace "data_volume_summaries_template.ipynb" with the name of the noteobook if you changed the notebook's nanme. 

## (the notebook name is the title of the notebook, so the report looks nicer if you change the name to something better)

In [ ]:
import sys
#Use this cell if you've moved this notebook somewhere else
#sys.path.insert(0, "/path/to/repo/beiwe/code")

In [ ]:
import data_summaries

In [ ]:
kr = data_summaries.read_keyring("/Volumes/SanDisk/onnela_lab/keyring_files/keyring_studies.py")

In [ ]:
data_summaries_file_path = "data_volume.csv"

In [ ]:
import os
study_id =  "58de9c9246b9fc61b2007034"
data_summaries.get_data_summaries(study_id,
        output_file_path = data_summaries_file_path,
        keyring = kr)

# Data Volume Summaries for Study

Here are the number of users with at least one day of data by data stream

In [ ]:
print(data_summaries.get_num_users(summaries_path = data_summaries_file_path).set_index("Data Type", drop = True))

## Summary Plots: X axis is time since study entry

In [ ]:
data_plots_path = "data_volume_plots"
data_summaries.data_volume_plots(
        data_summaries_path = data_summaries_file_path, 
        output_dir = data_plots_path,
        display_plots = True, #this needs to be true for the notebook to run
        binary_heatmap = True, #if this is False, a continuous data volume measurement will be put on the heatmaps
        plot_study_time = True, #whether to use study time instead of calendar time
        overlay_surveys = True, #whether to overlay survey submissions on top of data
        include_y_labels = False # if you have a ton of users, don't include y labels so you can fit it on one page. 
)

## Summary Plots: X axis is calendar time

In [ ]:
data_plots_path = "data_volume_plots"
data_summaries.data_volume_plots(
        data_summaries_path = data_summaries_file_path, 
        output_dir = data_plots_path,
        display_plots = True, 
        binary_heatmap = True, 
        plot_study_time = False,
        overlay_surveys = True,
        include_y_labels = False
)